# Imports

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from matplotlib.animation import FuncAnimation
from functools import partial
import itertools
import matplotlib.animation as animation
import os
from IPython.display import clear_output
import json

# Class Vector with all functions


In [15]:
class Vector:

    # angle between 0 and 360
    # length positive float max value = 10
    # speed negative or positive float between -1 and 1
    def __init__(self, angle, length, speed) -> None:
        assert angle >= 0   and angle <= 360
        assert length <= 10 and length > 0
        assert speed >= -1  and speed <= 1
        self.angle  = angle
        self.length = length
        self.speed  = speed

        self.endpoint = self.getEndpoint()
        self.radius   = self.getRadius()

        self.origin = [0, 0]

    def getEndpoint(self) -> list:
        angle = self.angle * math.pi / 180; 
        x = self.length * math.sin(angle)
        y = self.length * math.cos(angle)
        return [x, y]

    def getRadius(self) -> float:
        return math.sqrt(self.endpoint[0]*self.endpoint[0] + self.endpoint[1]*self.endpoint[1])

    # FIXME speed has to be adjusted
    def getTimedEndpoint(self, time) -> list:
        fixedSpeed = 10
        angleIncrement = fixedSpeed * self.speed * time
        angle = ((self.angle + angleIncrement) % 360) * math.pi / 180; 
        x = self.length * math.sin(angle)
        y = self.length * math.cos(angle)
        return [x, y]
    
    def setOrigin(self, origin):
        self.origin = origin

# Class for a datapoint

In [16]:
class Datapoint():

    def __init__(self, time) -> None:
        self.time = time
        self.points = None
        self.vectors = []

    def addVector(self, vector):
        self.vectors.append(vector)
    
    def setPoints(self, points):
        self.points = points

    def getTime(self):
        return self.time
    
    def getPoints(self):
        return self.points
    
    def getVectors(self):
        return self.vectors

# Creating n instances of Vector in a list


In [20]:
def createVectors(n, datapoint) -> list:
    vectors = []
    for i in range(n):

        # initializing random angle, length and speed

        angleNormalize = random.random()
        angle  = angleNormalize * 360

        lengthNormalize = random.random()
        length = lengthNormalize * 10
        if length == 0:
            length = 1

        speedNormalize = random.random()
        speed  = (speedNormalize - .5) * 2

        newVector = Vector(angle, length, speed)

        vectors.append(newVector)

        datapoint.addVector([lengthNormalize, angleNormalize, speedNormalize])

    return vectors

# getting the end points

In [21]:
# getting end of all vectors
def getEnd(vectors, t):
    end = np.array([0, 0])
    for i in vectors:
        end = end + np.array(i.getTimedEndpoint(t))
    return end

# loop through n frames and return all endpoints
def getAllEnds(vectors, n):
    ends = []
    for i in range(n):
        ends.append(getEnd(vectors, i))
    return ends

# getting the data set

In [26]:
vectorAmount = 10
amount       = 10

filenamePrefix = f"dataVectors{vectorAmount}Datapoints{amount}-"
filenameSuffixStart = 0
notFound = False

while (not notFound):
    if (os.path.isfile(f"./data/{filenamePrefix}{filenameSuffixStart}.json")):
        filenameSuffixStart += 1
    else:
        notFound = True

filename = f"./data/{filenamePrefix}{filenameSuffixStart}.json"

with open(filename, 'w') as f:
    dictEmpty = {}
    jsonObject = json.dumps(dictEmpty, indent=4)
    f.write(jsonObject)

for i in range(amount):

    time = random.randint(100, 1000)
    newDatapoint = Datapoint(time)

    vectors = createVectors(vectorAmount, newDatapoint)
    points  = getAllEnds(vectors, time)

    newDatapoint.setPoints(points)

    dataDict = {
        "points" : newDatapoint.getPoints(),
        "vectors" : newDatapoint.getVectors(),
        "time" : newDatapoint.getTime()
    }

    with open(filename, 'r+') as f:
        data = json.load(f)

        if(bool(data)):
            maxID = max([int(x) for x in data.keys()])+1
        else:
            maxID = 0
        
        data[str(maxID)] = dataDict
        f.seek(0)
        json.dump(data, f, indent=4)
        f.truncate()

    clear_output(wait=True)
    print(f"datapoint \#{i}")

0


TypeError: Object of type ndarray is not JSON serializable